In [1]:
from utils import get_opt_model
weights_path = '/home/llajan/OPT-175B-HF'
model, device_map = get_opt_model(weights_path, fp16=True)

In [7]:
prompt = "List down the key steps for making coffee."

In [ ]:
from transformers import AutoTokenizer

# Inference
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-30b')
inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(inputs["input_ids"].to(0), max_length=100)

print(tokenizer.decode(output[0].tolist()))